In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('../data/all_data.csv')
df

,payload,length,is_obf,label
0,c/ caridad s/n,14,0,0
1,"campello , el",12,0,0
2,40184,5,0,0
3,1442431887503330,16,0,0
4,nue37,5,0,0
...,...,...,...,...
60307,"170-169' and extractvalue 8981-999 , concat 37...",563,1,1
60308,"409-408"" order by 810-809-- OR 954-431=1452-17...",959,1,1
60309,380-379' procedure analyse extractvalue 6547-7...,977,1,1
60310,875-874 rlike select * from select sleep 276-2...,100,1,1


In [3]:
benign = df.loc[df.label == 0]
malicious = df.loc[df.label == 1]

In [5]:
data = benign.append(malicious.iloc[5])

from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse

count_vect = CountVectorizer(analyzer='word', ngram_range=(3,3))
raw_counts = count_vect.fit_transform(data['payload'])

# Append metadata to sparse matrix that will be used to train the classifier
num_feats = data[['length']].values
train_data = sparse.hstack((raw_counts, num_feats))
print(len(num_feats))

from sklearn.ensemble import IsolationForest

score = IsolationForest(random_state = 42).fit(train_data).decision_function(train_data)
ranked_index = np.argsort(score) 
malicious_rank = ranked_index.tolist().index(19304)
malicious_rank

38609


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:237: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)


19309

In [6]:
obf_malicious = malicious.loc[malicious.is_obf == 1]
non_obf_malicious = malicious.loc[malicious.is_obf == 0]

In [10]:
data

,payload,length,is_obf,label
0,c/ caridad s/n,14,0,0
1,"campello , el",12,0,0
2,40184,5,0,0
3,1442431887503330,16,0,0
4,nue37,5,0,0
...,...,...,...,...
56637,esdrs OR 591-533=1871-823 AND 457-451=501-495 ...,593,1,0
56638,violy belcastro OR 483-146=1662-906 OR 710-114...,435,1,0
56639,celeste_bryon@skullride.ki AND 547-547=941-941...,323,1,0
56640,98970543w AND 4=4 AND 7=7 OR 857=1191 OR 663=1...,270,1,0


In [11]:
import warnings
warnings.filterwarnings('ignore')

samples = 5
ranks = []

for i in range(samples):
    data = benign.append(obf_malicious.iloc[i])
    count_vect = CountVectorizer(analyzer='word', ngram_range=(3,3))
    raw_counts = count_vect.fit_transform(data['payload'])
    
    lengths = data[['length']].values
    train_data = sparse.hstack((raw_counts, lengths))
    
    score = IsolationForest(random_state = 42).fit(train_data).decision_function(train_data)
    ranked_index = np.argsort(score) 
    malicious_rank = ranked_index.tolist().index(19304)
    ranks.append(malicious_rank)
    print(malicious_rank)

19307
19304
38608
38608
38608


In [12]:
import warnings
warnings.filterwarnings('ignore')

samples = 5
ranks = []

for i in range(samples):
    data = benign.append(non_obf_malicious.iloc[i])
    count_vect = CountVectorizer(analyzer='word', ngram_range=(3,3))
    raw_counts = count_vect.fit_transform(data['payload'])
    
    lengths = data[['length']].values
    train_data = sparse.hstack((raw_counts, lengths))
    
    score = IsolationForest(random_state = 42).fit(train_data).decision_function(train_data)
    ranked_index = np.argsort(score) 
    malicious_rank = ranked_index.tolist().index(19304)
    ranks.append(malicious_rank)
    print(malicious_rank)

19305
19305
19305
19310
19306
